In [1]:
import math
from collections import defaultdict, Counter

# =========================
# CONFIG
# =========================

# Change these paths to where you saved the .conll files
TRAIN_PATH = "ar_padt-ud-train.conll"
DEV_PATH   = "ar_padt-ud-dev.conll"   # not strictly needed for basic version
TEST_PATH  = "ar_padt-ud-test.conll"

UNK_TOKEN = "<UNK>"
SMOOTHING = 1.0  # Laplace smoothing


# =========================
# DATA LOADING
# =========================

def read_conll(path):
    """
    Reads a simple CoNLL file: word[TAB]tag, sentences separated by blank lines.
    Returns a list of sentences, each sentence is a list of (word, tag) pairs.
    """
    sentences = []
    current = []

    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                if current:
                    sentences.append(current)
                    current = []
                continue
            parts = line.split("\t")
            if len(parts) != 2:
                # skip malformed lines
                continue
            word, tag = parts
            current.append((word, tag))

    if current:
        sentences.append(current)

    return sentences


def build_vocab(train_sentences, min_freq=2):
    """
    Build vocabulary from training sentences.
    Words with frequency < min_freq become UNK.
    Returns:
      - vocab: set of known words (excluding UNK)
      - word2freq: full frequency dictionary
    """
    freq = Counter()
    for sent in train_sentences:
        for word, _ in sent:
            freq[word] += 1

    vocab = {w for w, c in freq.items() if c >= min_freq}
    return vocab, freq


def replace_rare_with_unk(sentences, vocab):
    """
    Replace words not in vocab with UNK_TOKEN.
    Returns new list of sentences.
    """
    new_sents = []
    for sent in sentences:
        new_sent = []
        for word, tag in sent:
            if word not in vocab:
                new_sent.append((UNK_TOKEN, tag))
            else:
                new_sent.append((word, tag))
        new_sents.append(new_sent)
    return new_sents


# =========================
# HMM TRAINING
# =========================

class HMMPOSTagger:
    def __init__(self, smoothing=1.0):
        self.smoothing = smoothing
        self.tags = []
        self.tag_to_idx = {}
        self.idx_to_tag = {}
        self.vocab = set()

        # counts
        self.tag_counts = Counter()
        self.initial_tag_counts = Counter()
        self.transition_counts = defaultdict(Counter)  # prev_tag -> Counter(next_tag)
        self.emission_counts = defaultdict(Counter)    # tag -> Counter(word)

        # probabilities (log)
        self.log_initial = {}
        self.log_transition = defaultdict(dict)
        self.log_emission = defaultdict(dict)

    def fit(self, sentences):
        """
        sentences: list of sentences, each sentence is list of (word, tag).
        """

        # Count tags, transitions, emissions
        for sent in sentences:
            if not sent:
                continue

            # initial tag
            first_tag = sent[0][1]
            self.initial_tag_counts[first_tag] += 1

            prev_tag = None
            for word, tag in sent:
                self.tag_counts[tag] += 1
                self.emission_counts[tag][word] += 1
                self.vocab.add(word)

                if prev_tag is not None:
                    self.transition_counts[prev_tag][tag] += 1
                prev_tag = tag

        # list of unique tags
        self.tags = sorted(self.tag_counts.keys())
        self.tag_to_idx = {t: i for i, t in enumerate(self.tags)}
        self.idx_to_tag = {i: t for t, i in self.tag_to_idx.items()}

        num_tags = len(self.tags)
        vocab_size = len(self.vocab) + 1  # +1 for UNK if needed

        # Initial probabilities P(tag at position 0)
        total_initial = sum(self.initial_tag_counts.values())
        for tag in self.tags:
            count = self.initial_tag_counts[tag]
            # Laplace smoothing
            prob = (count + self.smoothing) / (total_initial + self.smoothing * num_tags)
            self.log_initial[tag] = math.log(prob)

        # Transition probabilities P(tag_i | tag_{i-1})
        for prev_tag in self.tags:
            total_prev = sum(self.transition_counts[prev_tag].values())
            for next_tag in self.tags:
                count = self.transition_counts[prev_tag][next_tag]
                prob = (count + self.smoothing) / (total_prev + self.smoothing * num_tags)
                self.log_transition[prev_tag][next_tag] = math.log(prob)

        # Emission probabilities P(word | tag)
        for tag in self.tags:
            total_tag = sum(self.emission_counts[tag].values())
            for word in self.vocab:
                count = self.emission_counts[tag][word]
                prob = (count + self.smoothing) / (total_tag + self.smoothing * vocab_size)
                self.log_emission[tag][word] = math.log(prob)

            # probability for UNK word for this tag
            unk_count = 0
            prob_unk = (unk_count + self.smoothing) / (total_tag + self.smoothing * vocab_size)
            self.log_emission[tag][UNK_TOKEN] = math.log(prob_unk)

    def viterbi(self, words):
        """
        Viterbi decoding to find best tag sequence for a list of words.
        words: list of tokens (strings).
        Returns: list of tags (strings).
        """
        T = len(words)
        N = len(self.tags)

        # Convert OOV words to UNK
        obs = [w if w in self.vocab else UNK_TOKEN for w in words]

        # dp[t][i] = max log prob of a path ending in tag i at position t
        dp = [[-math.inf] * N for _ in range(T)]
        backpointer = [[None] * N for _ in range(T)]

        # Initialization (t = 0)
        for i, tag in enumerate(self.tags):
            log_init = self.log_initial.get(tag, -math.inf)
            log_emit = self.log_emission[tag].get(obs[0], -math.inf)
            dp[0][i] = log_init + log_emit
            backpointer[0][i] = None

        # Recursion
        for t in range(1, T):
            for i, curr_tag in enumerate(self.tags):
                best_score = -math.inf
                best_prev = None
                log_emit = self.log_emission[curr_tag].get(obs[t], -math.inf)

                for j, prev_tag in enumerate(self.tags):
                    log_trans = self.log_transition[prev_tag].get(curr_tag, -math.inf)
                    score = dp[t-1][j] + log_trans + log_emit
                    if score > best_score:
                        best_score = score
                        best_prev = j

                dp[t][i] = best_score
                backpointer[t][i] = best_prev

        # Termination
        best_last_score = -math.inf
        best_last_idx = None
        for i in range(N):
            if dp[T-1][i] > best_last_score:
                best_last_score = dp[T-1][i]
                best_last_idx = i

        # Backtrack
        best_path_idx = [best_last_idx]
        for t in range(T-1, 0, -1):
            best_prev = backpointer[t][best_path_idx[-1]]
            best_path_idx.append(best_prev)
        best_path_idx.reverse()

        best_tags = [self.idx_to_tag[i] for i in best_path_idx]
        return best_tags

    def predict_sentence(self, sentence):
        """
        sentence: list of words
        returns: list of predicted tags
        """
        return self.viterbi(sentence)

    def evaluate(self, sentences):
        """
        sentences: list of sentences, each is list of (word, gold_tag)
        returns: accuracy
        """
        correct = 0
        total = 0

        for sent in sentences:
            words = [w for w, _ in sent]
            gold_tags = [t for _, t in sent]
            pred_tags = self.predict_sentence(words)

            for g, p in zip(gold_tags, pred_tags):
                if g == p:
                    correct += 1
                total += 1

        return correct / total if total > 0 else 0.0


# =========================
# MAIN
# =========================

def main():
    print("Loading data...")
    train_sents = read_conll(TRAIN_PATH)
    dev_sents = read_conll(DEV_PATH)
    test_sents = read_conll(TEST_PATH)

    print(f"#train sentences: {len(train_sents)}")
    print(f"#dev sentences:   {len(dev_sents)}")
    print(f"#test sentences:  {len(test_sents)}")

    # Build vocabulary from training data
    print("Building vocabulary...")
    vocab, freq = build_vocab(train_sents, min_freq=2)
    print(f"Vocab size (freq>=2): {len(vocab)}")

    # Replace rare / unknown words with UNK in train/dev/test
    train_sents_unk = replace_rare_with_unk(train_sents, vocab)
    dev_sents_unk   = replace_rare_with_unk(dev_sents, vocab)
    test_sents_unk  = replace_rare_with_unk(test_sents, vocab)

    print("Training HMM POS tagger...")
    tagger = HMMPOSTagger(smoothing=SMOOTHING)
    tagger.fit(train_sents_unk)
    print("Training done.")

    print("Evaluating on test set...")
    acc = tagger.evaluate(test_sents_unk)
    print(f"Test Accuracy: {acc:.4f}")

    # Show some example predictions
    print("\nExample predictions:")
    for sent in test_sents[:5]:
        words = [w for w, _ in sent]
        gold  = [t for _, t in sent]
        pred  = tagger.predict_sentence(words)
        print("Sentence: ", " ".join(words))
        print("GOLD:     ", " ".join(gold))
        print("PRED:     ", " ".join(pred))
        print("-" * 40)


if __name__ == "__main__":
    main()


Loading data...
#train sentences: 6075
#dev sentences:   909
#test sentences:  680
Building vocabulary...
Vocab size (freq>=2): 11886
Training HMM POS tagger...
Training done.
Evaluating on test set...
Test Accuracy: 0.8960

Example predictions:
Sentence:  سوريا : تعديل وزاري واسع يشمل 8 حقائب
GOLD:      X PUNCT NOUN ADJ ADJ VERB NUM NOUN
PRED:      X PUNCT NOUN NOUN ADJ VERB NUM NOUN
----------------------------------------
Sentence:  دمشق ( وكالات الانباء ) - اجرى الرئيس السوري بشار الاسد تعديلا حكومياً واسعا تم ب موجب ه إقالة وزيري الداخلية و الإعلام عن منصبي ها في حين ظل محمد ناجي العطري رئيساً ل الحكومة .
GOLD:      X PUNCT NOUN NOUN PUNCT PUNCT VERB NOUN ADJ X X NOUN ADJ ADJ VERB ADP NOUN PRON NOUN NOUN ADJ CCONJ NOUN ADP NOUN PRON ADP ADP VERB X X X NOUN ADP NOUN PUNCT
PRED:      NOUN PUNCT NOUN NOUN PUNCT PUNCT VERB NOUN ADJ X X X PUNCT CCONJ VERB ADP NOUN PRON NOUN NOUN ADJ CCONJ NOUN ADP NOUN PRON ADP NOUN NOUN X X X X ADP NOUN PUNCT
----------------------------------------
S